In [1]:
!pip install pyspark

  Using cached pyspark-3.2.4-py2.py3-none-any.whl
  Using cached py4j-0.10.9.5-py2.py3-none-any.whl (199 kB)


In [22]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [4]:
#creating spark session
spark = SparkSession.builder.master('local').appName('New Spark').getOrCreate()

/home/labuser/PycharmProjects/Spark/venv/lib/python3.6/site-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


In [20]:
#loading the nework data
net_df = spark.read.csv('/home/labuser/Downloads/networks.csv',header=True) 
net_df.show()

+---------+-----------+------------+
|series_id|series_info|     network|
+---------+-----------+------------+
|      101|       null|     netflix|
|      102|       null|amazon_prime|
|      103|       null|     appletv|
|      104|       null|     netflix|
|      105|       null|         HBO|
|      106|       null|         HBO|
|      107|       null|amazon_prime|
|      108|       null|     netflix|
|      109|       null|         ted|
|      110|       null|         sky|
+---------+-----------+------------+


In [17]:
ser_df = spark.read.csv('/home/labuser/Downloads/series.csv',header=True)
ser_df.show()

+---------+-------------+---------------+------+
|series_id|  series_name|          genre|rating|
+---------+-------------+---------------+------+
|      101| rickandmorty|          scifi|   9.1|
|      102|   invincible|       animated|   8.7|
|      103|forallmankind|historicalscifi|   7.9|
|      104|  blackmirror|  thrillerscifi|   8.0|
|      105|    westworld|          scifi|   8.1|
+---------+-------------+---------------+------+


In [85]:
#Remove all the rows from 'network_csv' whose 'series_id' is not present
df = net_df.join(ser_df,'series_id')
df.show()


+---------+-----------+------------+-------------+---------------+------+
|series_id|series_info|     network|  series_name|          genre|rating|
+---------+-----------+------------+-------------+---------------+------+
|      101|       null|     netflix| rickandmorty|          scifi|   9.1|
|      102|       null|amazon_prime|   invincible|       animated|   8.7|
|      103|       null|     appletv|forallmankind|historicalscifi|   7.9|
|      104|       null|     netflix|  blackmirror|  thrillerscifi|   8.0|
|      105|       null|         HBO|    westworld|          scifi|   8.1|
+---------+-----------+------------+-------------+---------------+------+


In [119]:
#Serie_info column in networks empyty fill with by concating series_name,genere from series.csv

df2 = ser_df.withColumn('full name',concat_ws(' ','series_name','genre'))
comb = net_df.join(df2,'series_id')
output = comb.select('series_id',coalesce('series_info','full name').alias('series_info'),'network').show()


+---------+--------------------+------------+
|series_id|         series_info|     network|
+---------+--------------------+------------+
|      101|  rickandmorty scifi|     netflix|
|      102| invincible animated|amazon_prime|
|      103|forallmankind his...|     appletv|
|      104|blackmirror thril...|     netflix|
|      105|     westworld scifi|         HBO|
+---------+--------------------+------------+


In [71]:
#find the series which has highest rating and return the series id output should be in string
one  = ser_df.orderBy('rating',ascending=False)
highest = one.select('series_id').first()[0]
highest

'101'

In [121]:
#find the sum of the rating from series dataframe and return the total sum int
total = ser_df.agg(sum('rating').alias('TotalSum'))
t_int = int(total.select('TotalSum').first()[0])
t_int



41

In [80]:
total2 =  ser_df.agg(sum('rating').alias("Total_sum")) 
t_float = float(total2.select('Total_sum').first()[0])
t_float


41.8

In [88]:
#new df
from pyspark.sql.types import *
Schema = StructType
art_df = spark.read.csv('/home/labuser/Downloads/art.csv',header=True)
art_df.show()

+---------+--------+----------+-----------+----------+------------+
|articleId|artistId|      date|    content|totalSales|totalReviews|
+---------+--------+----------+-----------+----------+------------+
|      501|     100|2023-01-02| Painting 1|       100|          50|
|      502|     200|2023-01-05| Painting 2|       130|         100|
|      503|     300|2023-01-08| Painting 3|       120|          40|
|      504|     100|2023-01-15| Painting 4|       140|          50|
|      505|     200|2023-01-18| Painting 5|       120|          90|
|      506|     300|2023-01-20| Painting 6|       110|          30|
|      507|     100|2023-02-01| Painting 7|       115|          70|
|      508|     200|2023-02-03| Painting 8|       125|          60|
|      509|     300|2023-02-10| Painting 9|       135|          80|
+---------+--------+----------+-----------+----------+------------+


In [91]:
#filter the articles by dataframe to only include the art pieces that were create between start_date and end_date

start_date = '2023-01-02'
end_date = '2023-02-10'

range_between = art_df.filter(art_df['date'].between(start_date,end_date)).show()

+---------+--------+----------+-----------+----------+------------+
|articleId|artistId|      date|    content|totalSales|totalReviews|
+---------+--------+----------+-----------+----------+------------+
|      501|     100|2023-01-02| Painting 1|       100|          50|
|      502|     200|2023-01-05| Painting 2|       130|         100|
|      503|     300|2023-01-08| Painting 3|       120|          40|
|      504|     100|2023-01-15| Painting 4|       140|          50|
|      505|     200|2023-01-18| Painting 5|       120|          90|
|      506|     300|2023-01-20| Painting 6|       110|          30|
|      507|     100|2023-02-01| Painting 7|       115|          70|
|      508|     200|2023-02-03| Painting 8|       125|          60|
|      509|     300|2023-02-10| Painting 9|       135|          80|
+---------+--------+----------+-----------+----------+------------+


In [123]:
#top n results of sales sum return in list

total = art_df.groupBy('artistId').agg(sum('totalSales').alias('totalSales'))
t2 = total.orderBy('artistId',ascending=False).limit(3)
t2_list = t2.select('artistId').rdd.map(lambda x : x[0])
t2_list.collect()

['300', '200', '100']

In [118]:
#average review as per the artist Return the total in float

avg_art = art_df.groupBy('artistId').agg(avg('totalReviews'))
avg_float = avg_art.select('avg(totalReviews)').first()[0]
avg_float

83.33333333333333